In [1]:
import os
import pandas as pd
import fnmatch

In [2]:
# NOTE: Possible names for each sequence

# "3D_T1": ['*SAG*T1*NII', '*SAG*FSPGR*3D*NII']
    

In [213]:
def get_plane(info):
    if 'SAG' in info: return 'SAG'
    elif 'COR' in info: return 'COR'
    elif 'AX' in info: return 'AX'
    else: return 'UNKNOWN'

In [272]:
def is_3D(info):
    if ('3D' or 'CUBE') in info:
        return True
    else:
        return False

In [273]:
def get_DOE(info):
#     print(info[-1])
    if 'I' in info[-1]:
        DOE=info[-2]
    else:
        DOE=info[-1]
    YYYY=DOE[0:4]
    MM=DOE[4:6]
    DD=DOE[6:8]
    return(YYYY+"/"+MM+'/'+DD)

In [274]:
def get_sequence_info(sequence_name):
    return (sequence_name.replace('-','_').replace('.','_').split('_'))[:-1]

In [295]:
def sort_T1(file_name):
    
    sequence_name=''
    sequence_DOE=''
    problem_T1=[]
    
    sequence_info=get_sequence_info(file_name)
    
    ID=sequence_info[0]
    
    if is_3D(sequence_info) and (get_plane(sequence_info) == 'SAG'):
        sequence_DOE=get_DOE(sequence_info)
        sequence_name='3D_FSPGR'
    
    #the scan is 2D
    elif (not is_3D(sequence_info)) and (get_plane(sequence_info) is not 'UNKNOWN'):
#         problem_T1.append(file_name)
        sequence_name=get_plane(sequence_info)+'_FSPGR'
        sequnece_DOE=get_DOE(sequence_info)

    else:
        problem_T1.append(file_name)
        
    print(ID+": ", sequence_name + " --> " +sequence_DOE)
        
    return(ID, [sequence_name, sequence_DOE])
    
#TO-DO: repeated exam of the same date or at the different date
    

In [296]:
def get_path(root):
    for path, subdirs, files in os.walk(root):
        for file in files:
            file_upper=file.upper()
            if fnmatch.fnmatch(file_upper, '*T1*NII') or fnmatch.fnmatch(file_upper, '*FSPGR*NII'):
#                 print("T1:", file_upper)
                file_info=sort_T1(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*FLAIR*NII'): 
#                 print("FLAIR: \t", file_upper)
#                 sort_FLAIR(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*T2*NII'):
#                 print('\t', file_upper)
#                 sort_T2(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*BOLD*NII'):
#                 sort_bold(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*DTI*NII') or fnmatch.fnmatch(file_upper, '*DKI*NII') or fnmatch.fnmatch(file_upper, '*DIFF*NII') or fnmatch.fnmatch(file_upper, '*DWI*NII') or fnmatch.fnmatch(file_upper, '*AVER*DC*NII') or fnmatch.fnmatch(file_upper, '*FRACT*ANISO*NII'):
#                 print('\t', file_upper)
#                 sort_diffusion(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*ASL*NII'):
#                 sort_asl(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*SWAN*NII'):
#                 sort_swan(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*CSF*NII'):
#                 sort_csf(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*CAROTID*NII'):
                pass
#                 print(file_upper)
#                 sort_carotid(file_upper)
            elif fnmatch.fnmatch(file_upper, '*AORTA*NII'):
#                 print(file_upper)
#                 sort_aorta(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*CEREBR*BLOOD*FLOW*NII'):
#                 print(file_upper)
#                 sort_cbf(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*MPR*NII'):
#                 print(file_upper)
#                 sort_mpr(file_upper)
                pass
            elif 'LOC' not in file_upper and fnmatch.fnmatch(file_upper, '*CANDY_CANE*NII'):
#                 print(file_upper)
#                 sort_candy_cane(file_upper)
                pass
            elif 'LOC' not in file_upper and fnmatch.fnmatch(file_upper, '*ARCH*NII'):
#                 print(file_upper)
#                 sort_arch(file_upper)
                pass
            elif 'LOC' not in file_upper and fnmatch.fnmatch(file_upper, '*PROBE*NII'):
#                 print(file_upper)
#                 sort_probe(file_upper)
                pass
            elif fnmatch.fnmatch(file_upper, '*NII'):
#                 print('\t', file_upper)
                pass
            else:
#                 print('\t\t', file_upper)
                pass


In [297]:
def main():
#     source_dir=input("Enter source dir:")
    source_dir="/Users/baymac/Desktop/Project_NaCl_Test_DataSet/3T_Last_updated_31OCT19KW"
    get_path(source_dir)
        

In [298]:
if __name__ =="__main__":
    main()

WH1720:  Unknow_FSPGR --> 
WH1720:  Unknow_FSPGR --> 
WH1720:  Unknow_FSPGR --> 
WH1720:  Unknow_FSPGR --> 
WH1720:  Unknow_FSPGR --> 
WH1720:  Unknow_FSPGR --> 
WH1720:  3D_FSPGR --> 2019/09/23
WH1720:  Unknow_FSPGR --> 
WH1648:  AX_FSPGR --> 
WH1648:  AX_FSPGR --> 
WH1648:  COR_FSPGR --> 
WH1648:  COR_FSPGR --> 
WH1648:  Unknow_FSPGR --> 
WH1648:  3D_FSPGR --> 2017/12/13
WH1707:  Unknow_FSPGR --> 
WH1707:  Unknow_FSPGR --> 
WH1707:  AX_FSPGR --> 
WH1707:  AX_FSPGR --> 
WH1707:  SAG_FSPGR --> 
WH1340:  AX_FSPGR --> 
WH1340:  AX_FSPGR --> 
WH1340:  Unknow_FSPGR --> 
WH1340:  Unknow_FSPGR --> 
WH1340:  3D_FSPGR --> 2019/04/04
WH1653:  AX_FSPGR --> 
WH1653:  AX_FSPGR --> 
WH1653:  COR_FSPGR --> 
WH1653:  COR_FSPGR --> 
WH1653:  Unknow_FSPGR --> 
WH1653:  3D_FSPGR --> 2018/02/13
WH1743:  Unknow_FSPGR --> 
WH1743:  Unknow_FSPGR --> 
WH1743:  Unknow_FSPGR --> 
WH1743:  Unknow_FSPGR --> 
WH1635:  AX_FSPGR --> 
WH1635:  AX_FSPGR --> 
WH1635:  Unknow_FSPGR --> 
WH1635:  Unknow_FSPGR --> 
WH163

WH1609:  AX_FSPGR --> 
WH1609:  AX_FSPGR --> 
WH1609:  COR_FSPGR --> 
WH1609:  COR_FSPGR --> 
WH1609:  3D_FSPGR --> 2017/06/23
WH1268:  AX_FSPGR --> 
WH1268:  AX_FSPGR --> 
WH1268:  COR_FSPGR --> 
WH1268:  COR_FSPGR --> 
WH1268:  3D_FSPGR --> 2017/08/30


In [33]:
def sort_FLAIR(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
    if ("SAG" or "SAGITTAL") in sequence_name:
        print(sequence_info)
    elif ('AX' or 'AXIAL') in sequence_name:
        print(sequence_info)
    elif ('COR' or 'CORONAL') in sequence_name:
        print(sequence_info)
    else: 
        print("\t ", sequence_name)

#TO-DO: T1 has 3D, 2D

In [34]:
def sort_T2(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
    if ("SAG" or "SAGITTAL") in sequence_name:
        print(sequence_info)
    elif ('AX' or 'AXIAL') in sequence_name:
        print(sequence_info)
    elif ('COR' or 'CORONAL') in sequence_name:
        print(sequence_info)
    else: 
        print("\t ", sequence_name)

#TO-DO: T2 has difference pulse types :0 and some other things

In [35]:
def sort_diffusion(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

#TO-DO: this one will be A LOT OF FUN!

In [36]:
def sort_bold(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

In [37]:
def sort_asl(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

In [38]:
def sort_csf(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

#TO-DO: CSF of different location and CINE ON/OFF

In [39]:
def sort_carotid(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

In [40]:
def sort_aorta(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

In [41]:
def sort_candy_cane(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

In [42]:
def sort_arch(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

In [43]:
def sort_mpr(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

In [44]:
def sort_cbf(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)

In [45]:
def sort_probe(sequence_name):
    sequence_info=get_sequence_info(sequence_name)
#     print(sequence_name)
    print(sequence_info)